In [1]:
import requests
from bs4 import BeautifulSoup as bs
from bs4 import BeautifulSoup, NavigableString, Tag

def scrape_and_parse(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 Gecko/20100101 Firefox/109.0',
        'Accept-Encoding': 'gzip, deflate, br'

    }
    html = requests.get(url , headers = headers)

    if html.status_code!=200:
        print('wrong'+url)
        return 'error'
    return bs(html.text, 'html.parser')

In [2]:
def get_show_info(link):
    url = 'https://www.imdb.com/title/'+link
    soup = scrape_and_parse(url)
    sum_url = url+'/plotsummary'
    sum_soup = scrape_and_parse(sum_url)

    section_class = ['div', 'ipc-chip-list--baseAlt ipc-chip-list']
    type_class = ['span', 'ipc-chip__text']

    summ_class = ['div', 'data-testid', 'sub-section-summaries']
    content_class = ['li','role', 'presentation']
    
    show_info = []

    types=[]

    section = soup.find(section_class[0], class_=section_class[1])
    types_soup = section.find_all(type_class[0], class_=type_class[1])
    for each in types_soup:
        types.append(each.get_text())
    
    summ = sum_soup.find(summ_class[0], {summ_class[1]: summ_class[2]})
    content = summ.find(content_class[0], {content_class[1]: content_class[2]}).get_text()
    show_info = [link, types, content]
    return show_info

In [3]:
import pandas as pd

links = pd.read_csv('l100.csv')['link']
infos = []
for link in links:
    l = get_show_info(link)
    infos.append(l)

In [6]:
movie_info = pd.DataFrame(columns=['link', 'types', 'summary'])
movie_info.head()

,link,types,summary


In [12]:
for i in range(0, 100):
    movie_info.loc[i, 'link'] = infos[i][0]
    movie_info.loc[i, 'types'] = infos[i][1]
    movie_info.loc[i, 'summary'] = infos[i][2]

In [13]:
movie_info.to_csv('worst_info.csv')